In [2]:
import pandas as pd
import numpy as np
import time
import re
import pickle
import time

In [3]:
tracking2018 = pd.read_csv("csv/tracking2018.csv")
tracking2018

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12777346,2018-09-11T05:25:17.800,123.18,51.73,3.76,1.45,0.29,NaN,NaN,None,NaN,football,NaN,NaN,football,59,2018091001,4033,right
12777347,2018-09-11T05:25:17.900,123.40,51.89,3.19,3.02,0.28,NaN,NaN,None,NaN,football,NaN,NaN,football,60,2018091001,4033,right
12777348,2018-09-11T05:25:18.000,123.55,52.18,2.95,3.88,0.32,NaN,NaN,None,NaN,football,NaN,NaN,football,61,2018091001,4033,right
12777349,2018-09-11T05:25:18.100,123.67,52.50,3.01,4.53,0.34,NaN,NaN,None,NaN,football,NaN,NaN,football,62,2018091001,4033,right


In [4]:
tracking2019 = pd.read_csv("csv/tracking2019.csv")
tracking2020 = pd.read_csv("csv/tracking2020.csv")

In [5]:
'''
Store all the desired plays
 - Was returned
 - No pass
 - No penalty yards
'''

play_data = pd.read_csv("csv/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull()", engine='python', inplace=False)

returned_plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
14,2018090600,2599,"(10:22) M.Bosher punts 46 yards to PHI 28, Cen...",3,4,16,ATL,Punt,Return,37267.0,...,NaN,NaN,NaN,3,6,NaN,46.0,9.0,37,36
26,2018090900,400,J.Tucker kicks 61 yards from BAL 35 to BUF 4. ...,1,0,0,BAL,Kickoff,Return,39470.0,...,NaN,NaN,NaN,7,0,NaN,61.0,28.0,33,45
27,2018090900,485,(7:25) (Punt formation) C.Bojorquez punts 46 y...,1,4,14,BUF,Punt,Return,46903.0,...,NaN,NaN,NaN,7,0,NaN,46.0,-1.0,47,82
28,2018090900,575,(6:37) (Punt formation) S.Koch punts 63 yards ...,1,4,9,BAL,Punt,Return,31018.0,...,NaN,NaN,NaN,7,0,NaN,63.0,4.0,59,36


In [6]:
def get_distances(x, player_pos):
    
    #print(f"x: {x}")
    #print(f"Player pos: {player_pos}")
    
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [ ]:
#################
### Cell that takes in 
#################

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue

    #####################

    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)

    # get starting time of play
    starting_time = min(play_df['time'])

    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)

            #print(f"time index: {time_index}")

            # check to make sure there is next state
            null_rewards = time_play_df.query("reward.isna()", engine='python', inplace=False)
            if (len(null_rewards) > 0):
                continue


            ball_carrier_pos = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y']]
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_pos,))
            time_play_df.sort_values("dist_from_ball_carrier", inplace=True, ignore_index=True)

            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']

            if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
                continue

            states = time_play_df.loc[:,['x','y']].to_numpy().reshape(-1)

            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y']].to_numpy().reshape(-1)
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue
            # can derive action by subtracting next position from current position
            action = states[0:2] - next_state[0:2]
            rowDict['action'] = action.round(2)

            dict_list.append(rowDict)
        except:
            print("random error")
            time.sleep(5)
            
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("all_players.pkl")

print(F"total time: {total_time}  seconds")

In [ ]:
#print(full_train_df)
#full_train_df.to_pickle('test.pkl')

new_df = pd.read_pickle('all_players.pkl')

print('new df')
print(type(new_df))
print(new_df.head())
print(new_df.shape)

'''
#full_train_df
with open('test.pkl', 'wb') as file:
    pickle.dump(full_train_df, file)

with open('all_players.pkl', 'rb') as file:
    full_df = pickle.load(file)
    
full_df
'''

In [ ]:
##################
### Get all stats, relative to first player
##################

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue

    #####################
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
#     play_df.loc[:, 'third_x'] = play_df.groupby('displayName').x.shift(-2)
#     play_df.loc[:, 'third_y'] = play_df.groupby('displayName').y.shift(-2)
#     play_df.loc[:, 'third_s'] = play_df.groupby('displayName').s.shift(-2)
#     play_df.loc[:, 'third_a'] = play_df.groupby('displayName').a.shift(-2)
#     play_df.loc[:, 'third_o'] = play_df.groupby('displayName').o.shift(-2)
#     play_df.loc[:, 'third_dir'] = play_df.groupby('displayName').dir.shift(-2)

    # get starting time of play
    starting_time = min(play_df['time'])

    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            #print(f"time index: {time_index}")

            # check to make sure there is next state
            null_rewards = time_play_df.query("reward.isna()", engine='python', inplace=False)
            if (len(null_rewards) > 0):
                continue

            # get distance from ball carrier
            ball_carrier_pos = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_pos.loc[:, ['x','y']],))
            
            time_play_df.sort_values("dist_from_ball_carrier", inplace=True, ignore_index=True)
            
            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']

            if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
                continue

            states = time_play_df.loc[:,['x','y', 's', 'a', 'o', 'dir']].to_numpy().reshape(-1)

            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir']].to_numpy().reshape(-1)
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue
            # can derive action by subtracting next position from current position
            action = states[0:2] - next_state[0:2]
            rowDict['action'] = action.round(2)
            
            #third_state = time_play_df.loc[:,['third_x','third_y', 'third_s', 'third_a', 'third_o', 'third_dir']].to_numpy().reshape(-1)
            #next_action = next_state[0:2] - third_state[0:2]
            #rowDict['next_action'] = next_action.round(2)

            dict_list.append(rowDict)
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)
            
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
        
    if index == 10:
            break
    
full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("datasets/all_players_rel_all_actions.pkl")

print(F"total time: {total_time}  seconds")

In [ ]:
full_train_df.iloc[0:4,:]

In [ ]:
##################
### Get all stats, relative to first player, groupby team
### Ball carrier's team (+ football) in first 12 slots, opponents in later slots
##################

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    

    # get starting time of play
    starting_time = min(play_df['time'])

    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            #print(f"time index: {time_index}")

            # check to make sure there is next state
            null_rewards = time_play_df.query("reward.isna()", engine='python', inplace=False)
            if (len(null_rewards) > 0):
                continue
            
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            other_team_name = 'home'
            if ball_carrier_team == 'home':
                other_team_name = 'away'

            '''
            # break into two teams
            ball_carrier_team = time_play_df.loc[time_play_df.team == ball_carrier_team, :]
            other_team = time_play_df.loc[time_play_df.team == other_team_name, :]
            '''
            
            # get distance from ball carrier
            ball_carrier_pos = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_pos.loc[:, ['x','y']],))
            
            #print(time_play_df.team[0,0])
            
            time_play_df['team_index'] = time_play_df.team == other_team_name
            
            #pd.Series([x == other_team_name for x in time_play_df.team.values])
                      
            time_play_df.sort_values(by=["team_index","dist_from_ball_carrier"], inplace=True, ignore_index=True)
            
            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']
                

            time_play_df.loc[football_index,'o'] = 0
            time_play_df.loc[football_index,'dir'] = 0
            
            if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
                continue

            states = time_play_df.loc[:,['x','y', 's', 'a', 'o', 'dir']].to_numpy().reshape(-1)

            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir']].to_numpy().reshape(-1)
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue
            # can derive action by subtracting next position from current position
            action = states[0:2] - next_state[0:2]
            rowDict['action'] = action.round(2)
            
            #third_state = time_play_df.loc[:,['third_x','third_y', 'third_s', 'third_a', 'third_o', 'third_dir']].to_numpy().reshape(-1)
            #next_action = next_state[0:2] - third_state[0:2]
            #rowDict['next_action'] = next_action.round(2)

            dict_list.append(rowDict)
            
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)

            
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
        
    
full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("datasets/all_players_rel_all_actions_group_team.pkl")

print(F"total time: {total_time}  seconds")

try


In [13]:
### Get all stats, relative to first player, groupby team
### Ball carrier's team (+ football) in first 12 slots, opponents in later slots
### Save entire next state (x,y,s,a,dir, theta)
##################

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

for index in range(0, num_plays):

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    # if no returner id, don't include play
    if pd.isnull(returnerId):
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    

    # get starting time of play
    starting_time = min(play_df['time'])

    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            #print(f"time index: {time_index}")

            # check to make sure there is next state
            null_rewards = time_play_df.query("reward.isna()", engine='python', inplace=False)
            if (len(null_rewards) > 0):
                continue
            
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            other_team_name = 'home'
            if ball_carrier_team == 'home':
                other_team_name = 'away'

            '''
            # break into two teams
            ball_carrier_team = time_play_df.loc[time_play_df.team == ball_carrier_team, :]
            other_team = time_play_df.loc[time_play_df.team == other_team_name, :]
            '''
            
            # get distance from ball carrier
            ball_carrier_pos = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    
            time_play_df['dist_from_ball_carrier'] = time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_pos.loc[:, ['x','y']],))
            
            #print(time_play_df.team[0,0])
            
            time_play_df['team_index'] = time_play_df.team == other_team_name
            
            #pd.Series([x == other_team_name for x in time_play_df.team.values])
                      
            time_play_df.sort_values(by=["team_index","dist_from_ball_carrier"], inplace=True, ignore_index=True)
            
            #### 
            #Check to make sure ball carrier has football
            #   i.e. 'football' isn't second row in df and distance is less than hyperparameter
            ####
            football_index = time_play_df.index[time_play_df.team == 'football']
            try:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier'].iloc[0] 
            except:
                football_distance = time_play_df.loc[football_index, 'dist_from_ball_carrier']
                

            time_play_df.loc[football_index,'o'] = 0
            time_play_df.loc[football_index,'dir'] = 0
            
            if (football_index != 1) | (football_distance > FOOTBALL_PLAYER_DIST):
                continue

            states = time_play_df.loc[:,['x','y', 's', 'a', 'o', 'dir']].to_numpy().reshape(-1)

            rowDict['season'] = season
            rowDict['gameId'] = gameId
            rowDict['playId'] = playId
            
            rowDict["playIndex"] = index
            rowDict["timeIndex"] = time_index
            # add current state
            rowDict['state'] = states
            # add future states
            next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir']].to_numpy().reshape(-1)
            rowDict['next_state'] = next_state
            # add reward
            try:
                rowDict['reward'] = time_play_df.loc[0,'reward']
            except:
                print("error getting reward")
                continue
            # can derive action by subtracting next position from current position
            action = states[0:6]- next_state[0:6]
            rowDict['action'] = action.round(2)
            
            #third_state = time_play_df.loc[:,['third_x','third_y', 'third_s', 'third_a', 'third_o', 'third_dir']].to_numpy().reshape(-1)
            #next_action = next_state[0:2] - third_state[0:2]
            #rowDict['next_action'] = next_action.round(2)

            dict_list.append(rowDict)
            
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)

            
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
        
    
full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

full_train_df = full_train_df.dropna(axis=0, how='any', inplace=False)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("datasets/all_players_rel_all_actions_group_team_six_action_state.pkl")

print(F"total time: {total_time}  seconds")

Play 0 out of 4494
Play 250 out of 4494
Could not get returner id for index 362 at game 2018100701 for play 4281: <class 'Exception'>
Play 500 out of 4494
Could not get returner id for index 502 at game 2018101410 for play 2172: <class 'Exception'>
Play 750 out of 4494
Play 1000 out of 4494
Could not get returner id for index 1096 at game 2018120212 for play 1876: <class 'Exception'>
Play 1250 out of 4494
Could not get returner id for index 1470 at game 2018123011 for play 36: <class 'Exception'>
Play 1500 out of 4494
Play 1750 out of 4494
Play 2000 out of 4494
Play 2250 out of 4494
Could not get returner id for index 2443 at game 2019112408 for play 3869: <class 'Exception'>
Play 2500 out of 4494
Could not get returner id for index 2736 at game 2019121512 for play 4390: <class 'Exception'>
Play 2750 out of 4494
Play 3000 out of 4494
Play 3250 out of 4494
Play 3500 out of 4494
random error: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
r

In [ ]:

'''
State vector is 
'''

def return_state(gameId, time, df):
    state_df = df.query('time == @time and gameId == @gameId')
    
    
    
    return state_df

In [ ]:
time = tracking2018.iloc[100,0]
print(time)
gameId = tracking2018.loc[0,'gameId']
print(F"game i: {gameId}")

#return_state(gameId, time, tracking2018)
max_frame = max(tracking2018.query("playId == 36 and gameId == @gameId")['frameId'])
print(max_frame)
tracking2018.query("frameId == 11 and playId == 36 and gameId == @gameId")

In [ ]:
'''
Cell to run to save training set, where consider all states and players. Ball carrier listed first, football last
'''

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)
#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
        
        time_play_df = play_df.query("time == @time_string", inplace=False)
        
        # sort df to put ball carrier first, football last
        time_play_df['sort'] = pd.Categorical(time_play_df['nflId'], [returnerId])
        time_play_df.sort_values("sort", na_position='last', inplace=True)
        

        states = time_play_df.loc[:,['x','y', 's', 'a', 'o', 'dir']].to_numpy().reshape(-1)
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)

        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = states[0:2]
        football_pos = states[-6:-4]
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        # add current state
        rowDict['state'] = states
        # add future states
        next_state = time_play_df.loc[:,['next_x','next_y', 'next_s', 'next_a', 'next_o', 'next_dir']].to_numpy().reshape(-1)
        rowDict['next_state'] = next_state
        # add reward
        rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        # can derive action by subtracting next position from current position
        action = states[0:6] - next_state[0:6]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

full_train_df.to_csv("all_actions_all_players.csv")


In [ ]:
'''
Cell to run to save training set, where consider (x,y) states for all players. Ball carrier listed first, football last
'''

pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)

#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    # get proper CSV depending on the year
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    # Define next states
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
           
        time_play_df = play_df.query("time == @time_string", inplace=False)
        time_play_df.query("not reward.isnull()", engine='python', inplace=True)
        
        # sort df to put ball carrier first, football last
        time_play_df['sort'] = pd.Categorical(time_play_df['nflId'], [returnerId])
        time_play_df.sort_values("sort", na_position='last', inplace=True)
        

        states = time_play_df.loc[:,['x','y']].to_numpy().reshape(-1)
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)

        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = states[0:2]
        football_pos = states[-2:]
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        rowDict["playIndex"] = index
        rowDict["timeIndex"] = time_index
        # add current state
        rowDict['state'] = states
        # add future states
        next_state = time_play_df.loc[:,['next_x','next_y']].to_numpy().reshape(-1)
        rowDict['next_state'] = next_state
        # add reward
        try:
            rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        except:
            print("error getting reward")
            print(ball_carrier_entry)
            continue
        # can derive action by subtracting next position from current position
        action = states[0:2] - next_state[0:2]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
full_train_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

full_train_df.to_csv("all_players.csv")

In [ ]:
saved = pd.read_csv('all_players.csv', index_col=0)
saved.head()

In [ ]:
saved.query("reward.isnull()", engine='python', inplace=False)

In [ ]:
'''
Cell to run to save training set, where only consider ball-carrier and football as states. No other players.
States also shortened to only x,y positions
'''


pd.options.mode.chained_assignment = None

start_time = time.time()

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
FOOTBALL_PLAYER_DIST = 1.7
num_plays = len(returned_plays)
#for each play
for index in range(0, num_plays):
    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
    
    if pd.isnull(returnerId):
        print(f"Returner id is null is for index {index} at game {gameId} for play {playId}")
        continue
    
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'reward'] = play_df.groupby('displayName').dis.shift(-1)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    
    all_time_steps = play_df.loc[:,'time'].unique()
    
    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):
        # now we can define the values for each row
        rowDict = {}
        
        time_play_df = play_df.query("time == @time_string", inplace=False)
        
        
        # get expected ball carrier
        ball_carrier_entry = time_play_df.query("nflId == @returnerId", inplace=False)
        football_entry = time_play_df.query("displayName == 'football'", inplace=False)
        
        ########
        ## Check to make sure ball returner has the ball. If not, skip
        ########
        ball_carrier_pos = ball_carrier_entry.loc[:,['x','y']].to_numpy()
        football_pos = football_entry.loc[:,['x','y']].to_numpy()
        carrier_football_dist = round(np.linalg.norm(ball_carrier_pos - football_pos),3)
        if carrier_football_dist > FOOTBALL_PLAYER_DIST:
            continue
            
        if (len(football_pos) == 0) or (len(ball_carrier_pos) == 0):
            continue
            
        # add current state
        current_state = np.zeros((1, 4))
        current_state[0, 0:2] = ball_carrier_pos
        current_state[0,2:4] = football_pos
        rowDict['state'] = current_state
        # add future states
        next_state = np.zeros((1, 4))
        next_state[0,0:2] = ball_carrier_entry.loc[:,['next_x','next_y']]
        next_state[0,2:4] = football_entry.loc[:,['next_x','next_y']]
        rowDict['next_state'] = next_state
        # add reward
        rowDict['reward'] = ball_carrier_entry.loc[:,'reward'].iloc[0]
        # can derive action by subtracting next position from current position
        action = current_state[0, 0:2] - next_state[0,0:2]
        rowDict['action'] = action.round(2)
        
        dict_list.append(rowDict)
        
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
simple1_training_df = pd.DataFrame(dict_list)

end_time = time.time()
total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

simple1_training_df.to_csv("simple1_train.csv")
